In [1]:
import pandas as pd

df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

lista_colunas = list(df_test.columns)
lista_colunas.append('IN_TREINEIRO')

df = df[lista_colunas]
df.head(3)

,NU_INSCRICAO,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047,IN_TREINEIRO
0,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,43,RS,24,M,1,1,1,4,1,...,520.0,D,D,C,A,A,C,H,A,0
1,2c3acac4b33ec2b195d77e7c04a2d75727fad723,23,CE,17,F,3,1,2,0,2,...,580.0,A,A,B,A,A,A,NaN,A,0
2,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,23,CE,21,F,3,1,3,0,1,...,NaN,D,D,C,A,A,A,NaN,A,0


In [2]:
print(f"Percentagem de Treineiros (Treineiros {df['IN_TREINEIRO'].sum()}, DataFrame {df.shape[0]})")
round(df['IN_TREINEIRO'].sum()/df.shape[0], 2)

Percentagem de Treineiros (Treineiros 1783, DataFrame 13730)


0.13

In [3]:
df.drop(['TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC', 'Q027', 'SG_UF_RESIDENCIA',  'TP_PRESENCA_MT'], axis=1, inplace=True)

In [4]:
df.dropna(inplace=True)

In [5]:
print(f"Percentagem de Treineiros (Treineiros {df['IN_TREINEIRO'].sum()}, DataFrame {df.shape[0]})")
round(df['IN_TREINEIRO'].sum()/df.shape[0], 2)

Percentagem de Treineiros (Treineiros 1498, DataFrame 10097)


0.15

In [6]:
#df_1 = df[df['IN_TREINEIRO'] == 1]
#df_0 = df[df['IN_TREINEIRO'] == 0].sample(frac=0.9, random_state=42)

#df = pd.concat([df_1, df_0])

## Encoders

In [7]:
lista_encoder = ['TP_SEXO', 'Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047']

In [8]:
from sklearn.preprocessing import LabelEncoder
import pickle as pkl

for i in lista_encoder:
    le = LabelEncoder()
    le.fit(df[i])
    df[i] = le.transform(df[i])
    
    file = open(f'./encoder/encoder_{i}.pkl', 'wb')
    pkl.dump(le, file)
    file.close()

In [9]:
df['NU_NOTA_CN'].fillna(df['NU_NOTA_CN'].mean(), inplace=True)
df['NU_NOTA_CH'].fillna(df['NU_NOTA_CH'].mean(), inplace=True)
df['NU_NOTA_LC'].fillna(df['NU_NOTA_CH'].mean(), inplace=True)

## Predicted

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, roc_curve, confusion_matrix
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

x = df.drop(['NU_INSCRICAO', 'IN_TREINEIRO'], axis=1)
y = df['IN_TREINEIRO']

In [11]:
scale = StandardScaler()
scale.fit(x)
x = scale.transform(x)

file = open(f'./encoder/scale.pkl', 'wb')
pkl.dump(scale, file)
file.close()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1/3, random_state=42)

In [12]:
from sklearn.svm import SVC

lr = LogisticRegression()
#lr = SVC()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)

In [13]:
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
print(confusion_matrix(y_pred, y_test))

0.9928698752228164
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      2832
           1       0.99      0.97      0.98       534

    accuracy                           0.99      3366
   macro avg       0.99      0.98      0.99      3366
weighted avg       0.99      0.99      0.99      3366

[[2826    6]
 [  18  516]]


## Test

In [14]:
filename = './models/lr.sav'
pkl.dump(lr, open(filename, 'wb'))

In [ ]:
df.drop(['NU_INSCRICAO', 'IN_TREINEIRO'], axis=1).columns

0    4517
1      53
Name: IN_TREINEIRO, dtype: int64
